# 13.영화 추천 엔진 만들기

In [2]:
# MovieLens data 가져오기
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("source/u.data", sep="\t", header=None)
df.columns = ["user_id", "item_id", "rating", "timestamp"]

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))

for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)
ratings_train.shape, ratings_test.shape

((631, 1682), (312, 1682))

### 영화의 수를 k로 사용해 영화 간 유사도 행렬 계산

In [6]:
from sklearn.neighbors import NearestNeighbors

k = ratings_train.shape[1]      # 영화의 수
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

In [7]:
neigh.fit(ratings_train.T)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=1682, p=2,
                 radius=1.0)

In [8]:
# 유사도(거리)
item_distances, _ = neigh.kneighbors(ratings_train.T, return_distance=True)

In [9]:
item_distances

array([[0.        , 0.28112785, 0.30634744, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.34340416, 0.36248159, ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.53423537, 0.54586079, ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.        , 0.4452998 , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.0513167 , 0.50763404, ..., 1.        , 1.        ,
        1.        ]])

In [10]:
item_distances.shape

(1682, 1682)

### 평가 예측 및 모델의 성능 측정

In [11]:
item_pred = ratings_train.dot(item_distances) / np.array([np.abs(item_distances).sum(axis=1)])

In [13]:
item_pred.shape

(631, 1682)

In [14]:
item_pred

array([[6.43138819e-17, 2.68346602e-01, 2.73377633e-01, ...,
        5.76694411e-01, 5.91031787e-01, 5.89962397e-01],
       [4.17758507e-17, 1.70392459e-01, 1.73586594e-01, ...,
        3.31747919e-01, 3.39995605e-01, 3.39380431e-01],
       [5.07105702e-17, 3.27208145e-01, 3.32466794e-01, ...,
        6.26634958e-01, 6.42213921e-01, 6.41051924e-01],
       ...,
       [8.85422654e-18, 6.34129164e-02, 6.25449872e-02, ...,
        1.02259215e-01, 1.04801513e-01, 1.04611889e-01],
       [3.29216241e-17, 1.66859982e-01, 1.67810294e-01, ...,
        3.23424495e-01, 3.31465250e-01, 3.30865509e-01],
       [1.55351429e-17, 2.53346239e-02, 2.63412738e-02, ...,
        6.24256837e-02, 6.39776677e-02, 6.38619090e-02]])